<a href="https://colab.research.google.com/github/DikshantBadawadagi/Animal-Injury-Analysis-LLama-3.2-11B/blob/main/Connect-Llama-to-flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 4.8 MB/s eta 0:00:00


In [4]:
from groq import Groq
import os
os.environ["GROQ_API_KEY"] = "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"

In [ ]:
api_key = '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'

In [11]:
client = Groq()
messages = []
for image_path in image_paths:
  print(image_path)
  base64_image = encode_image(image_path)
  chat_completion = client.chat.completions.create(

      messages=[
          {
              "role": "user",
              "content": [
                  {
                      "type": "text",
                      "text": "What is this image"
                  },
                  {
                      "type": "image_url",
                      "image_url":{
                          "url": f"data:image/jpeg;base64,{base64_image}",
                      }
                  }
              ]
          },
          {
              "role": "assistant",
              "content": ""
          }
      ],
      model="llama-3.2-11b-vision-preview",
      temperature=0,
      max_tokens=1024,
      top_p=1,
      stream=False,
      stop=None,

  )
  a = chat_completion.choices[0].message.content
  print(a)

/content/image1.jpeg
The image is a close-up photograph of a tabby cat, showcasing its distinctive striped fur pattern and large ears.
/content/image2.jpeg
This image is a close-up photograph of a chameleon, showcasing its vibrant blue skin and striking red eyes. The chameleon is perched on a pink flower petal, with yellow stamens visible in the background. The image is likely intended to highlight the chameleon's unique features and colors, and may be used for educational or informational purposes.


In [10]:
from groq import Groq
import base64

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

image_paths = [
    "/content/image1.jpeg",
    "/content/image2.jpeg"
]

In [7]:
!pip install flask_ngrok
!pip install --upgrade pyngrok

ngrok_token = "2s47sYhNtjWqVTyGx8Fegvw7EXD_9So3TYkSwfm3Ln7sh2as"
from pyngrok import ngrok
!ngrok authtoken ngrok_token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [11]:
!ngrok config add-authtoken 2s47sYhNtjWqVTyGx8Fegvw7EXD_9So3TYkSwfm3Ln7sh2as

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [14]:
from flask import Flask,render_template, request, jsonify
from groq import Groq
import os
import base64


app = Flask(__name__)
client = Groq()

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

@app.route('/')
def index():
  return render_template('/content/index.html')

@app.route('/upload',methods=['POST'])
def process_image():
  if 'image' not in request.files:
    return jsonify({"error": "No image uploaded"}),400
  image = request.files['image']
  image_path = os.path.join("uploads", image.filename)
  image.save(image_path)
  base64_image = encode_image(image_path)

  try:
    chat_completion = client.chat.completions.create(

      messages=[
          {
              "role": "user",
              "content": [
                  {
                      "type": "text",
                      "text": "What is this image"
                  },
                  {
                      "type": "image_url",
                      "image_url":{
                          "url": f"data:image/jpeg;base64,{base64_image}",
                      }
                  }
              ]
          },
          {
              "role": "assistant",
              "content": ""
          }
      ],
      model="llama-3.2-11b-vision-preview",
      temperature=0,
      max_tokens=1024,
      top_p=1,
      stream=False,
      stop=None,

    )
    a = chat_completion.choices[0].message.content
    os.remove(image_path)
    print(a)
    return jsonify(a)
  except Exception as e:
    os.remove(image_path)
    return jsonify({"error": str(e)}),500


if __name__ == "__main__":
  public_url = ngrok.connect(5000)
  os.makedirs("uploads", exist_ok=True)
  print(f"Public URL: {public_url}")

  app.run(port=5000)

Public URL: NgrokTunnel: "https://45a7-34-16-150-76.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:__main__:Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
          